# **Setup**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp '/content/drive/MyDrive/AI4D_Malawi_News_Classification/data/Train.csv' .
!cp '/content/drive/MyDrive/AI4D_Malawi_News_Classification/data/Test.csv' .

# **Libraries**

In [ ]:
import pandas as pd 
from sklearn.model_selection import StratifiedKFold
import numpy as np 
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
import cloudpickle
from google_drive_downloader import GoogleDriveDownloader as gdd
import warnings
warnings.filterwarnings("ignore")

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

In [ ]:
from keras.models import Sequential
from keras.layers.recurrent import LSTM, GRU
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.embeddings import Embedding
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils

from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from keras.preprocessing import sequence, text
from keras.callbacks import EarlyStopping

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedShuffleSplit
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Convolution1D, Dropout
from keras.optimizers import SGD
from keras.utils import np_utils

In [ ]:
import keras
from keras.layers import Concatenate , Input ,concatenate
from keras.models import Sequential, Model

In [ ]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers.recurrent import LSTM, GRU
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.embeddings import Embedding
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from keras.optimizers import Adam
from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from keras.preprocessing import sequence, text
from keras.callbacks import EarlyStopping
from keras.metrics import Accuracy

In [ ]:
from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV

In [ ]:
from sklearn.linear_model import LogisticRegression

# **Load Data**

In [ ]:
train = pd.read_csv('Train.csv') 

In [ ]:
test = pd.read_csv('Test.csv') 

In [ ]:
LABEL = 'Label'

In [ ]:
# Dowload pretrained vectorizer
file_id = '1F7q38t_KASJKLcs2IBdF4k0pbm1lgk5A'
destination = '/content/vectorizer.txt'
gdd.download_file_from_google_drive(file_id=file_id,
                                    dest_path=destination)

# **Modeling : ResNet**

In [ ]:
def CUSTOM_ResNet(shape1) :
  # define two sets of inputs
  input_1 = tf.keras.layers.Input(shape = (shape1),name='20_Features')
  input_conv = tf.keras.layers.Input(shape = (shape1, 1),name='20_Features_Conv')
  
  # create Convolution Layer 
  head_conv = Convolution1D(filters=64, kernel_size=1, input_shape=(shape1, 1),name='Conv_Layer')(input_conv)
  head_conv = Activation('relu',name='Conv_Activation')(head_conv)
  input_2_conv = Flatten()(head_conv)
  
  # create a simple 3 layers for input_1
  # Layer 1 -- input_1
  head_2 = tf.keras.layers.BatchNormalization(name='Conv_Features_BatchNormalization_1')(input_2_conv)
  head_2 = tf.keras.layers.Dense(512, activation = "relu",name='Conv_Features_Layer_1')(head_2)
  head_2 = tf.keras.layers.Dropout(0.2,name='Conv_Features_DropOut_1')(head_2)
  # Layer 2 -- input_1
  head_2 = tf.keras.layers.BatchNormalization(name='Conv_Features_BatchNormalization_2')(head_2)
  input_2 = tf.keras.layers.Dense(128, activation = "relu",name='Conv_Features_Output_Layer_2')(head_2)


  # create a simple 3 layers for input_1
  # Layer 1 -- input_1
  head_1 = tf.keras.layers.BatchNormalization(name='20_Features_BatchNormalization_1')(input_1)
  head_1 = tf.keras.layers.Dense(256, activation = "relu",name='20_Features_Layer_1')(head_1)
  head_1 = tf.keras.layers.Dropout(0.15,name='20_Features_DropOut_1')(head_1)
  # Layer 2 -- input_1
  head_1 = tf.keras.layers.BatchNormalization(name='20_Features_BatchNormalization_2')(head_1)
  input_3 = tf.keras.layers.Dense(128, activation = "relu",name='20_Features_Output_Layer_2')(head_1)

  input_4 = tf.keras.layers.Concatenate(name='Concatenate_Outputs_Features')([input_3,input_2])

  # create a simple 3 layers for input_4
  # Layer 1 -- input_1
  head_4 = tf.keras.layers.BatchNormalization(name='Conv_20_Features_BatchNormalization_1')(input_4)
  head_4 = tf.keras.layers.Dense(256, activation = "relu",name='Conv_20_Features_Layer_1')(head_4)
  input_5 = tf.keras.layers.Dropout(0.2,name='Conv_20_Features_DropOut_1')(head_4)

  # combine the output vector of the two branches
  input_4_avg = tf.keras.layers.Average()([input_5, input_4]) 

  head_3 = tf.keras.layers.BatchNormalization(name='Outputs_Features_BatchNormalization')(input_4_avg)
  head_3 = tf.keras.layers.Dense(256, "relu",name='Outputs_Features_Layer_1')(head_3)
  head_3 = tf.keras.layers.Dropout(0.15,name='Outputs_Features_DropOut_1')(head_3)
  
  head_3 = tf.keras.layers.BatchNormalization(name='Outputs_Features_BatchNormalization_2')(head_3)
  head_3 = tf.keras.layers.Dense(160, "relu",name='Outputs_Features_Layer_2')(head_3)
  
  head_3 = tf.keras.layers.Dense(80, "relu",name='Outputs_Features_Layer_3')(head_3)
  head_3 =  tf.keras.layers.Dropout(0.4,name='Outputs_Features_DropOut_3')(head_3)
  
  output = tf.keras.layers.Dense(20, activation = "softmax",name='Output')(head_3)
  
  model = tf.keras.models.Model(inputs = [input_1,input_conv], outputs = output)
  model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=[tf.keras.metrics.CategoricalAccuracy()])
  return model 

In [ ]:
# Function to seed everything
import random , os
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    tf.random.set_seed(seed)

In [ ]:
n_splits = 5
kf = StratifiedKFold(n_splits=n_splits, random_state=10, shuffle=True)

# custom vectorizer 
vectorizer = cloudpickle.load(open('vectorizer.txt', 'rb'))
X_train = vectorizer.predict(train['Text'])
X_test = vectorizer.predict(test['Text'])


# reshape test data
X_test_r = np.zeros((len(X_test), 20, 1))
X_test_r[:, :, 0] = X_test[:, :]


y_train = train[LABEL].copy()
n_labels = train[LABEL].unique().shape[0]

# we need to binarize the labels for the neural net
from sklearn.preprocessing import  LabelEncoder
LE = LabelEncoder()
ytrain_enc = pd.get_dummies(y_train).values
TARGETS = pd.get_dummies(y_train).columns

y_oof = np.zeros([X_train.shape[0], n_labels])
y_test = np.zeros([X_test.shape[0], n_labels])

i = 0
metrics = list()
apply_aug = False

for tr_idx, val_idx in kf.split(X_train, y_train):

    X_tr, X_vl = X_train[tr_idx, :], X_train[val_idx, :]
    y_tr, y_vl = ytrain_enc[tr_idx], ytrain_enc[val_idx]
    y_train_, y_vld_ = y_train[tr_idx], y_train[val_idx]

    # reshape train data
    X_train_r = np.zeros((len(X_tr), 20, 1))
    X_train_r[:, :, 0] = X_tr[:, :]

    # reshape validation data
    X_valid_r = np.zeros((len(X_vl), 20, 1))
    X_valid_r[:, :, 0] = X_vl[:, :]
    
    # 1 CNN ANN
    seed_everything(seed=1)
    model_ResNet = CUSTOM_ResNet(shape1=20)
    model_ResNet.fit([X_tr,X_train_r], y=y_tr, batch_size=8, epochs=8, verbose=0, validation_data=([X_vl,X_valid_r], y_vl))
    y_pred_ResNet = model_ResNet.predict([X_vl,X_valid_r])
    test_pred_ResNet = model_ResNet.predict([X_test,X_test_r])
    
    # 2 - MLP
    model1 = MLPClassifier(250, random_state=47) 
    model1.fit(X_tr, y_train_)
    y_pred1    = model1.predict_proba(X_vl)
    test_pred1 = model1.predict_proba(X_test)
    
    y_pred = (y_pred1+y_pred_ResNet ) / 2.0
    y_oof[val_idx, :] = y_pred
    y_tr, y_vl = y_train.iloc[tr_idx], y_train.iloc[val_idx]
    metric = accuracy_score(y_vl, TARGETS[np.argmax(y_pred,axis=1)])
    print("fold #{} val_accuracy: {}".format(i, metric))
    
    i += 1
    test_pred = (test_pred1 + test_pred_ResNet ) / 2.0
    y_test += test_pred / n_splits
    metrics.append(metric)
    
metrics = np.array(metrics).mean()
print(f'Full accuracy {metrics}')  # Full accuracy 0.6908222028648858

fold #0 val_accuracy: 0.6979166666666666
fold #1 val_accuracy: 0.6968641114982579
fold #2 val_accuracy: 0.6968641114982579
fold #3 val_accuracy: 0.6898954703832753
fold #4 val_accuracy: 0.686411149825784
Full accuracy 0.6935903019744483


In [ ]:
# Prepare submission
y_sub = np.argmax(y_test, 1)
y_sub = pd.DataFrame({LABEL: y_sub})
class_ = TARGETS
y_sub[LABEL] = y_sub[LABEL].apply(lambda x: class_[x])
sub = test[['ID']]
sub['LABEL'] = y_sub[LABEL]
sub.head(15)

,ID,LABEL
0,ID_ADHEtjTi,SOCIAL ISSUES
1,ID_AHfJktdQ,RELIGION
2,ID_AUJIHpZr,RELATIONSHIPS
3,ID_AUKYBbIM,POLITICS
4,ID_AZnsVPEi,WILDLIFE/ENVIRONMENT
5,ID_AcEmtLNf,RELIGION
6,ID_AeOXSXRV,POLITICS
7,ID_AgyQFeQn,ECONOMY
8,ID_AjOcippL,POLITICS
9,ID_AjzKuqko,POLITICS


In [ ]:
sub.to_csv(f'seed_10_Resnet_MLP_sub_{round(metrics,4)}.csv', index=False)